In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
import librosa
import librosa.display
import IPython.display as ipd 
import os
import csv

In [26]:
def slice_dataframe(machine):
    """ Slice dataframe to get data for 6dB samples of abnormal and normal sounds."""
    df = pd.read_csv('anomaly_files.csv')
    df_machine_sounds_abnorm = df[(df.dB == f'6_dB_{machine}') & (df.state == 'abnormal')]
    df_machine_sounds_norm = df[(df.dB == f'6_dB_{machine}') & (df.state == 'normal')]
    
    return df_machine_sounds_abnorm, df_machine_sounds_norm

In [27]:
def extract_feature(file_path, target):
    """ Extract the following features: mfccs, delta_mfccs and delta2_mfccs."""
    X, sample_rate = librosa.load(file_path)

    feature_dict = {}
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T,axis=0)
    for idx, item in enumerate(mfccs):
        feature_dict[f'mfccs{idx}'] = item
    
    zero_cross =  librosa.feature.zero_crossing_rate(y=X, frame_length=2048, hop_length=512, center=True)
    for idx, item in enumerate(zero_cross[0]):
        feature_dict[f'zero_cross{idx}'] = item
        
    rms = librosa.feature.rms(y=X, S=None, frame_length=2048, hop_length=512, center=True, pad_mode='reflect')
    for idx, item in enumerate(rms[0]):
        feature_dict[f'rms{idx}'] = item
        
    #delta_mfccs = librosa.feature.delta(mfccs)
    #for idx, item in enumerate(delta_mfccs):
    #    feature_dict[f'delta_mfccs{idx}'] = item
        
    #delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    #for idx, item in enumerate(delta2_mfccs):
    #    feature_dict[f'delta2_mfccs{idx}'] = item
    feature_dict['target'] = target
   
    return feature_dict
#extract_feature('E:/Projects/factory_noises/6_dB_valve/valve/id_00/abnormal/00000037.wav', 1)

In [28]:
def get_features(df_path, target):
    """ Put extracted features into a dataframe."""
    features = []
    for row in df_path.file_path:
        features.append(extract_feature(row, target))
        print(f'Extracted features for {row}')
    df_features = pd.DataFrame(features)    
    return df_features    

In [29]:
def store_features(df_norm, df_abnorm, features_file_name):
    """ Save features into a .csv file."""
    df_features = pd.concat([df_norm, df_abnorm])
    df_features.to_csv(features_file_name)

In [30]:
def retrieve_and_store_feature(machine, features_file_name):
    """ Run all the functions to retrieve and store the features. Choose a machine and filename to store the information. """
    print('slicing dataframe')
    df_machine_sounds_abnorm, df_machine_sounds_norm = slice_dataframe(machine)
    
    print('getting features')
    df_norm = get_features(df_machine_sounds_norm, 1)
    df_abnorm = get_features(df_machine_sounds_abnorm, 0)
    
    print('storing features')
    store_features(df_norm, df_abnorm, features_file_name)

In [7]:
retrieve_and_store_feature('valve', 'df_6dB_valve_fe_ta_imb.csv')

slicing dataframe
getting features
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000000.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000001.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000002.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000003.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000004.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000005.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000006.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000007.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000008.wav
Extracted features for E:\Projects\factory_noises\6_dB_valve\valve\id_00\abnormal\00000009.wav
Extracted featu

KeyboardInterrupt: 

In [ ]:
retrieve_and_store_feature('pump', 'df_6dB_pump_fe_ta_imb.csv')

In [ ]:
retrieve_and_store_feature('slider', 'df_6dB_slider_fe_ta_imb.csv')

In [ ]:
retrieve_and_store_feature('fan', 'df_6dB_fan_fe_ta_imb.csv')